In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import io

# Configure the page
st.set_page_config(
    page_title="Hotel Booking Analyzer",
    page_icon="📍",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Custom CSS for modern styling
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        font-weight: 600;
        color: #1f2937;
        margin-bottom: 1rem;
        text-align: center;
    }
    .section-header {
        font-size: 1.5rem;
        font-weight: 500;
        color: #374151;
        margin: 2rem 0 1rem 0;
        border-bottom: 2px solid #e5e7eb;
        padding-bottom: 0.5rem;
    }
    .metric-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 1.5rem;
        border-radius: 12px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        text-align: center;
        color: white;
    }
    .metric-value {
        font-size: 2rem;
        font-weight: bold;
        margin-bottom: 0.5rem;
    }
    .metric-label {
        font-size: 0.9rem;
        opacity: 0.9;
    }
    .upload-section {
        background: #f8fafc;
        padding: 2rem;
        border-radius: 12px;
        border: 2px dashed #cbd5e1;
        margin: 2rem 0;
    }
    .info-box {
        background: linear-gradient(135deg, #f0f9ff 0%, #e0f2fe 100%);
        padding: 1.5rem;
        border-radius: 12px;
        border-left: 4px solid #0ea5e9;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

def analyze_booking_data(df):
    """Analyze booking data and return comprehensive statistics"""
    # Group by country and calculate metrics
    country_analysis = df.groupby('Hotel Country Name').agg({
        'Guests': ['count', 'mean', 'sum'],
        'Room Nights': ['mean', 'sum']
    }).round(2)
    
    # Flatten column names
    country_analysis.columns = ['Total_Bookings', 'Avg_Guests', 'Total_Guests', 'Avg_Stay_Nights', 'Total_Nights']
    country_analysis = country_analysis.reset_index()
    
    # Sort by total bookings for ranking
    country_analysis = country_analysis.sort_values('Total_Bookings', ascending=False)
    
    analysis = {
        'total_bookings': len(df),
        'total_countries': len(country_analysis),
        'total_guests': df['Guests'].sum(),
        'avg_stay': df['Room Nights'].mean(),
        'country_data': country_analysis,
        'top_country': country_analysis.iloc[0]['Hotel Country Name'] if not country_analysis.empty else 'N/A'
    }
    
    return analysis

def create_interactive_map(analysis):
    """Create modern interactive choropleth map with improved styling"""
    country_data = analysis['country_data']
    
    # Create the choropleth map with modern color scheme
    fig = go.Figure(data=go.Choropleth(
        locations=country_data['Hotel Country Name'],
        z=country_data['Total_Bookings'],
        locationmode='country names',
        colorscale=[
            [0, '#f8fafc'],
            [0.2, '#e2e8f0'],
            [0.4, '#94a3b8'],
            [0.6, '#64748b'],
            [0.8, '#475569'],
            [1, '#1e293b']
        ],
        autocolorscale=False,
        text=country_data['Hotel Country Name'],
        hovertemplate=
        '<b>%{text}</b><br>' +
        'Total Bookings: %{z:,}<br>' +
        'Average Guests: ' + country_data['Avg_Guests'].astype(str) + '<br>' +
        'Average Stay: ' + country_data['Avg_Stay_Nights'].astype(str) + ' nights<br>' +
        'Total Guests: ' + country_data['Total_Guests'].astype(str) +
        '<extra></extra>',
        colorbar=dict(
            title=dict(
                text="Total Bookings",
                font=dict(size=14, color='#374151')
            ),
            tickfont=dict(size=12, color='#6b7280'),
            thickness=15,
            len=0.8
        )
    ))

    fig.update_layout(
        title={
            'text': 'Global Hotel Booking Distribution',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 24, 'color': '#1f2937', 'family': 'Arial, sans-serif'}
        },
        geo=dict(
            showframe=False,
            showcoastlines=True,
            coastlinecolor='#e5e7eb',
            showland=True,
            landcolor='#f9fafb',
            showocean=True,
            oceancolor='#f0f9ff',
            projection_type='natural earth',
            showlakes=True,
            lakecolor='#dbeafe'
        ),
        height=650,
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    
    return fig

def create_metrics_cards(analysis):
    """Create modern metric cards"""
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown(f"""
        <div class="metric-container">
            <div class="metric-value">{analysis['total_bookings']:,}</div>
            <div class="metric-label">Total Bookings</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        st.markdown(f"""
        <div class="metric-container">
            <div class="metric-value">{analysis['total_countries']}</div>
            <div class="metric-label">Countries</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        st.markdown(f"""
        <div class="metric-container">
            <div class="metric-value">{analysis['total_guests']:,}</div>
            <div class="metric-label">Total Guests</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col4:
        st.markdown(f"""
        <div class="metric-container">
            <div class="metric-value">{analysis['avg_stay']:.1f}</div>
            <div class="metric-label">Avg Stay (nights)</div>
        </div>
        """, unsafe_allow_html=True)

def main():
    # Main header
    st.markdown('<h1 class="main-header">Hotel Booking Analytics Dashboard</h1>', unsafe_allow_html=True)
    
    # Info section
    st.markdown('<div class="info-box">', unsafe_allow_html=True)
    st.markdown("""
    **Data Requirements:** Your CSV file should contain exactly 3 columns:
    - **Hotel Country Name** - Country where booking was made
    - **Guests** - Number of guests for each booking  
    - **Room Nights** - Number of nights stayed
    """)
    st.markdown('</div>', unsafe_allow_html=True)
    
    # File upload section
    st.markdown('<div class="upload-section">', unsafe_allow_html=True)
    st.markdown('<h3 class="section-header">Data Upload</h3>', unsafe_allow_html=True)
    uploaded_file = st.file_uploader(
        "Select your booking data CSV file to begin analysis",
        type=['csv'],
        help="Upload a CSV file with your hotel booking data"
    )
    st.markdown('</div>', unsafe_allow_html=True)
    
    if uploaded_file is not None:
        try:
            # Read and validate data
            df = pd.read_csv(uploaded_file)
            
            # Validate CSV structure
            expected_columns = ['Hotel Country Name', 'Guests', 'Room Nights']
            if len(df.columns) != 3:
                st.error(f'Invalid file format: Expected 3 columns, found {len(df.columns)}')
                return
            
            # Rename columns to expected format
            df.columns = expected_columns
            
            # Validate and clean data
            try:
                df['Guests'] = pd.to_numeric(df['Guests'], errors='coerce')
                df['Room Nights'] = pd.to_numeric(df['Room Nights'], errors='coerce')
            except:
                st.error('Data validation failed: Guests and Room Nights must be numeric values')
                return
            
            # Remove invalid data
            df = df.dropna()
            
            if df.empty:
                st.error('No valid data found after processing')
                return
            
            # Analyze data
            analysis = analyze_booking_data(df)
            
            # Success message
            st.success(f"Successfully processed {uploaded_file.name}")
            
            # Display metrics
            st.markdown('<br>', unsafe_allow_html=True)
            create_metrics_cards(analysis)
            
            # Interactive map
            st.markdown('<h3 class="section-header">Geographic Distribution</h3>', unsafe_allow_html=True)
            fig = create_interactive_map(analysis)
            st.plotly_chart(fig, use_container_width=True)
            
            # Top countries chart
            st.markdown('<h3 class="section-header">Top 10 Countries by Bookings</h3>', unsafe_allow_html=True)
            top_10 = analysis['country_data'].head(10)
            
            bar_fig = px.bar(
                top_10, 
                x='Total_Bookings', 
                y='Hotel Country Name',
                orientation='h',
                color='Total_Bookings',
                color_continuous_scale=[
                    [0, '#f1f5f9'],
                    [0.5, '#64748b'],
                    [1, '#1e293b']
                ],
                labels={'Total_Bookings': 'Total Bookings', 'Hotel Country Name': 'Country'}
            )
            
            bar_fig.update_layout(
                height=400,
                showlegend=False,
                plot_bgcolor='white',
                paper_bgcolor='white',
                font=dict(color='#374151'),
                yaxis=dict(categoryorder='total ascending')
            )
            
            st.plotly_chart(bar_fig, use_container_width=True)
            
            # Detailed statistics table
            st.markdown('<h3 class="section-header">Detailed Statistics</h3>', unsafe_allow_html=True)
            
            # Format the dataframe
            display_df = analysis['country_data'].copy()
            display_df = display_df.rename(columns={
                'Hotel Country Name': 'Country',
                'Total_Bookings': 'Total Bookings',
                'Avg_Guests': 'Avg Guests',
                'Total_Guests': 'Total Guests',
                'Avg_Stay_Nights': 'Avg Stay (nights)',
                'Total_Nights': 'Total Nights'
            })
            
            # Format numbers for better readability
            display_df['Total Bookings'] = display_df['Total Bookings'].apply(lambda x: f"{x:,}")
            display_df['Total Guests'] = display_df['Total Guests'].apply(lambda x: f"{x:,}")
            display_df['Total Nights'] = display_df['Total Nights'].apply(lambda x: f"{x:,}")
            
            # Download section
            _, col2, _ = st.columns([1, 1, 1])
            with col2:
                csv_buffer = io.StringIO()
                display_df.to_csv(csv_buffer, index=False)
                st.download_button(
                    label="Download Analysis Results",
                    data=csv_buffer.getvalue(),
                    file_name=f"booking_analysis_{uploaded_file.name}",
                    mime="text/csv",
                    use_container_width=True
                )
            
        except Exception as e:
            st.error(f'Error processing file: {str(e)}')

if __name__ == '__main__':
    main()


In [ ]:
# Convert 'Booking Month' to datetime for proper sorting
df['Month_Date'] = pd.to_datetime(df['Booking Month'], format='%b %Y')
df_sorted = df.sort_values('Month_Date')

# Calculate bookings by month and status with sorted data
bookings_summary = df_sorted.groupby(['Booking Month', 'Booking Status']).size().reset_index(name='Count')

# Create a pivot table with properly sorted months
pivot_table = bookings_summary.pivot(index='Booking Month', columns='Booking Status', values='Count').fillna(0)

# Sort the pivot table by converting index to datetime
month_order = pd.to_datetime(pivot_table.index, format='%b %Y').sort_values()
sorted_months = [month.strftime('%b %Y') for month in month_order]
pivot_table = pivot_table.reindex(sorted_months)

print("Bookings by Month and Status (Chronologically Sorted):")
print(pivot_table)

# Create visualizations with sorted data
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Stacked bar chart
pivot_table.plot(kind='bar', stacked=True, ax=axes[0,0], color=['lightcoral', 'lightblue', 'lightgreen'])
axes[0,0].set_title('Bookings by Month (Stacked)')
axes[0,0].set_xlabel('Month')
axes[0,0].set_ylabel('Number of Bookings')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Grouped bar chart
pivot_table.plot(kind='bar', ax=axes[0,1], color=['lightcoral', 'lightblue', 'lightgreen'])
axes[0,1].set_title('Bookings by Month (Grouped)')
axes[0,1].set_xlabel('Month')
axes[0,1].set_ylabel('Number of Bookings')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Line plot
pivot_table.plot(kind='line', ax=axes[1,0], marker='o', color=['red', 'blue', 'green'])
axes[1,0].set_title('Booking Trends by Month')
axes[1,0].set_xlabel('Month')
axes[1,0].set_ylabel('Number of Bookings')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Pie chart for total bookings by status
status_totals = df['Booking Status'].value_counts()
axes[1,1].pie(status_totals.values, labels=status_totals.index, autopct='%1.1f%%', 
              colors=['lightcoral', 'lightblue', 'lightgreen'])
axes[1,1].set_title('Overall Booking Status Distribution')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nTotal bookings: {len(df)}")
print(f"Confirmed bookings: {status_totals.get('Confirmed', 0)}")
print(f"Cancelled bookings: {status_totals.get('Cancelled', 0)}")
print(f"Rejected bookings: {status_totals.get('Rejected', 0)}")

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

# Configure page
st.set_page_config(
    page_title="Booking Analysis Dashboard",
    page_icon="📊",
    layout="wide"
)

def process_data(df):
    # Convert 'Booking Month' to datetime for proper sorting
    df['Month_Date'] = pd.to_datetime(df['Booking Month'], format='%b %Y')
    df_sorted = df.sort_values('Month_Date')

    # Calculate bookings by month and status with sorted data
    bookings_summary = df_sorted.groupby(['Booking Month', 'Booking Status']).size().reset_index(name='Count')

    # Create a pivot table with properly sorted months
    pivot_table = bookings_summary.pivot(index='Booking Month', columns='Booking Status', values='Count').fillna(0)

    # Sort the pivot table by converting index to datetime
    month_order = pd.to_datetime(pivot_table.index, format='%b %Y').sort_values()
    sorted_months = [month.strftime('%b %Y') for month in month_order]
    pivot_table = pivot_table.reindex(sorted_months)

    return pivot_table, df_sorted

def create_visualizations(pivot_table, df):
    # Modern color palette
    colors = {
        'Cancelled': '#FF8A95',
        'Confirmed': '#81C784',
        'Rejected': '#90CAF9'
    }

    # Set modern styling
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.size'] = 11

    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.patch.set_facecolor('#FAFBFC')

    # 1. Stacked bar chart
    pivot_table.plot(kind='bar', stacked=True, ax=axes[0,0], 
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.85, edgecolor='white', linewidth=1.5)
    axes[0,0].set_title('Monthly Bookings Distribution', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[0,0].set_xlabel('Month', fontsize=12, color='#34495E')
    axes[0,0].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[0,0].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[0,0].tick_params(axis='y', colors='#34495E')
    axes[0,0].set_facecolor('#FAFBFC')
    axes[0,0].grid(True, alpha=0.3, color='#BDC3C7')
    axes[0,0].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 2. Grouped bar chart
    pivot_table.plot(kind='bar', ax=axes[0,1], 
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.85, edgecolor='white', linewidth=1.5)
    axes[0,1].set_title('Monthly Booking Comparison', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[0,1].set_xlabel('Month', fontsize=12, color='#34495E')
    axes[0,1].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[0,1].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[0,1].tick_params(axis='y', colors='#34495E')
    axes[0,1].set_facecolor('#FAFBFC')
    axes[0,1].grid(True, alpha=0.3, color='#BDC3C7')
    axes[0,1].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 3. Line plot
    pivot_table.plot(kind='line', ax=axes[1,0], marker='o', markersize=8, linewidth=3,
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.9)
    axes[1,0].set_title('Booking Trends Over Time', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[1,0].set_xlabel('Month', fontsize=12, color='#34495E')
    axes[1,0].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[1,0].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[1,0].tick_params(axis='y', colors='#34495E')
    axes[1,0].set_facecolor('#FAFBFC')
    axes[1,0].grid(True, alpha=0.3, color='#BDC3C7')
    axes[1,0].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 4. Donut chart
    status_totals = df['Booking Status'].value_counts()
    pie_colors = [colors.get(status, '#CCCCCC') for status in status_totals.index]

    wedges, texts, autotexts = axes[1,1].pie(status_totals.values, labels=status_totals.index, 
                                             autopct='%1.1f%%', colors=pie_colors,
                                             wedgeprops=dict(width=0.5, alpha=0.85, edgecolor='white', linewidth=2),
                                             textprops={'fontsize': 11, 'color': '#2C3E50'},
                                             pctdistance=0.75, startangle=90)

    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(10)

    axes[1,1].set_title('Overall Status Distribution', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')

    # Add center circle for donut effect
    centre_circle = plt.Circle((0,0), 0.3, fc='#FAFBFC', linewidth=2, edgecolor='#ECF0F1')
    axes[1,1].add_artist(centre_circle)

    total_bookings = len(df)
    axes[1,1].text(0, 0, f'{total_bookings}\nTotal\nBookings', 
                   horizontalalignment='center', verticalalignment='center',
                   fontsize=12, fontweight='bold', color='#2C3E50')

    plt.tight_layout(pad=3.0)
    
    return fig

# Main app
st.title("📊 Booking Analysis Dashboard")
st.markdown("Upload your CSV file to analyze booking data")

# File upload
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    try:
        # Read CSV file
        df = pd.read_csv(uploaded_file)
        
        # Validate required columns
        required_columns = ['Booking Week', 'Booking Month', 'Booking Status']
        if not all(col in df.columns for col in required_columns):
            st.error(f'CSV file must contain columns: {", ".join(required_columns)}')
        else:
            # Process data
            pivot_table, df_processed = process_data(df)
            
            # Calculate summary statistics
            status_totals = df['Booking Status'].value_counts()
            total_bookings = len(df)
            
            # Display summary statistics
            st.header("📈 Summary Statistics")
            col1, col2, col3, col4 = st.columns(4)
            
            with col1:
                st.metric("Total Bookings", total_bookings)
            with col2:
                confirmed = status_totals.get('Confirmed', 0)
                confirmed_pct = (confirmed/total_bookings*100) if total_bookings > 0 else 0
                st.metric("Confirmed", f"{confirmed}", f"{confirmed_pct:.1f}%")
            with col3:
                cancelled = status_totals.get('Cancelled', 0)
                cancelled_pct = (cancelled/total_bookings*100) if total_bookings > 0 else 0
                st.metric("Cancelled", f"{cancelled}", f"{cancelled_pct:.1f}%")
            with col4:
                rejected = status_totals.get('Rejected', 0)
                rejected_pct = (rejected/total_bookings*100) if total_bookings > 0 else 0
                st.metric("Rejected", f"{rejected}", f"{rejected_pct:.1f}%")
            
            # Create and display visualizations
            st.header("📊 Visualizations")
            fig = create_visualizations(pivot_table, df_processed)
            st.pyplot(fig)
            
            # Display data table
            st.header("📋 Data Summary")
            st.dataframe(pivot_table, use_container_width=True)
            
    except Exception as e:
        st.error(f'Error processing file: {str(e)}')
else:
    st.info("Please upload a CSV file to get started")


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
from matplotlib.backends.backend_pdf import PdfPages

# Configure page
st.set_page_config(
    page_title="Booking Analysis Dashboard",
    page_icon="📊",
    layout="wide"
)

def process_data(df, time_period):
    time_column = 'Booking Week' if time_period == 'Week' else 'Booking Month'
    
    # Convert to datetime for proper sorting
    if time_period == 'Week':
        # Handle week format more flexibly
        df['Time_Date'] = pd.to_datetime(df[time_column], errors='coerce')
        if df['Time_Date'].isna().all():
            # Try alternative week formats
            df['Time_Date'] = pd.to_datetime(df[time_column].str.replace(r'Week\s+(\d+)\s+(\d+)', r'\2-W\1', regex=True), errors='coerce')
    else:
        df['Time_Date'] = pd.to_datetime(df[time_column], format='%b %Y', errors='coerce')
    
    # Drop rows with invalid dates
    df = df.dropna(subset=['Time_Date'])
    
    if df.empty:
        # Return empty pivot table with proper structure if no valid dates
        return pd.DataFrame(), df, time_column
        
    df_sorted = df.sort_values('Time_Date')

    # Calculate bookings by time period and status with sorted data
    bookings_summary = df_sorted.groupby([time_column, 'Booking Status']).size().reset_index(name='Count')

    # Create a pivot table with properly sorted time periods
    pivot_table = bookings_summary.pivot(index=time_column, columns='Booking Status', values='Count').fillna(0)

    # Sort the pivot table by the original time order
    if not pivot_table.empty:
        time_sorted_df = df_sorted.drop_duplicates(subset=[time_column]).sort_values('Time_Date')
        sorted_periods = time_sorted_df[time_column].tolist()
        pivot_table = pivot_table.reindex(sorted_periods, fill_value=0)

    return pivot_table, df_sorted, time_column

def create_visualizations(pivot_table, df, time_period):
    # Modern color palette
    colors = {
        'Cancelled': '#FF8A95',
        'Confirmed': '#81C784',
        'Rejected': '#90CAF9'
    }

    # Set modern styling
    try:
        plt.style.use('seaborn-v0_8-whitegrid')
    except OSError:
        plt.style.use('default')
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.size'] = 11

    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.patch.set_facecolor('#FAFBFC')

    period_label = time_period.title()

    # 1. Stacked bar chart
    pivot_table.plot(kind='bar', stacked=True, ax=axes[0,0], 
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.85, edgecolor='white', linewidth=1.5)
    axes[0,0].set_title(f'{period_label}ly Bookings Distribution', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[0,0].set_xlabel(period_label, fontsize=12, color='#34495E')
    axes[0,0].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[0,0].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[0,0].tick_params(axis='y', colors='#34495E')
    axes[0,0].set_facecolor('#FAFBFC')
    axes[0,0].grid(True, alpha=0.3, color='#BDC3C7')
    axes[0,0].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 2. Grouped bar chart
    pivot_table.plot(kind='bar', ax=axes[0,1], 
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.85, edgecolor='white', linewidth=1.5)
    axes[0,1].set_title(f'{period_label}ly Booking Comparison', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[0,1].set_xlabel(period_label, fontsize=12, color='#34495E')
    axes[0,1].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[0,1].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[0,1].tick_params(axis='y', colors='#34495E')
    axes[0,1].set_facecolor('#FAFBFC')
    axes[0,1].grid(True, alpha=0.3, color='#BDC3C7')
    axes[0,1].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 3. Line plot
    pivot_table.plot(kind='line', ax=axes[1,0], marker='o', markersize=8, linewidth=3,
                    color=[colors.get(col, '#CCCCCC') for col in pivot_table.columns], 
                    alpha=0.9)
    axes[1,0].set_title('Booking Trends Over Time', fontsize=14, fontweight='bold', pad=20, color='#2C3E50')
    axes[1,0].set_xlabel(period_label, fontsize=12, color='#34495E')
    axes[1,0].set_ylabel('Number of Bookings', fontsize=12, color='#34495E')
    axes[1,0].tick_params(axis='x', rotation=45, colors='#34495E')
    axes[1,0].tick_params(axis='y', colors='#34495E')
    axes[1,0].set_facecolor('#FAFBFC')
    axes[1,0].grid(True, alpha=0.3, color='#BDC3C7')
    axes[1,0].legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9)

    # 4. Donut chart
    status_totals = df['Booking Status'].value_counts()
    # 4. Donut chart
    status_totals = df['Booking Status'].value_counts()
    pie_colors = [colors.get(status, '#CCCCCC') for status in status_totals.index]

    _, _, autotexts = axes[1,1].pie(status_totals.values, labels=status_totals.index, 
                                             autopct='%1.1f%%', colors=pie_colors,
                                             wedgeprops=dict(width=0.5, alpha=0.85, edgecolor='white', linewidth=2),
                                             textprops={'fontsize': 11, 'color': '#2C3E50'},
                                             pctdistance=0.75, startangle=90)

    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(10)

    # Add center circle for donut effect
    centre_circle = plt.Circle((0,0), 0.3, fc='#FAFBFC', linewidth=2, edgecolor='#ECF0F1')
    axes[1,1].add_artist(centre_circle)

    total_bookings = len(df)
    axes[1,1].text(0, 0, f'{total_bookings}\nTotal\nBookings', 
                   horizontalalignment='center', verticalalignment='center',
                   fontsize=12, fontweight='bold', color='#2C3E50')

    plt.tight_layout(pad=3.0)
    
    return fig

def fig_to_base64(fig):
    """Convert matplotlib figure to base64 string for copying"""
    img_buffer = io.BytesIO()
    fig.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    img_buffer.seek(0)
    img_str = base64.b64encode(img_buffer.getvalue()).decode()
    return img_str

def create_pdf_download(fig):
    """Create PDF download button"""
    pdf_buffer = io.BytesIO()
    with PdfPages(pdf_buffer) as pdf:
        pdf.savefig(fig, bbox_inches='tight', dpi=300)
    pdf_buffer.seek(0)
    return pdf_buffer.getvalue()
st.title("📊 Booking Analysis Dashboard")
st.markdown("Upload your CSV file to analyze booking data")

# File upload
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    try:
        # Read CSV file
        df = pd.read_csv(uploaded_file)
        
        # Validate required columns
        required_columns = ['Booking Week', 'Booking Month', 'Booking Status']
        if not all(col in df.columns for col in required_columns):
            st.error(f'CSV file must contain columns: {", ".join(required_columns)}')
        else:
            # Time period selection
            st.header("⚙️ Analysis Settings")
            time_period = st.selectbox(
                "Select time period for analysis:",
                options=['Month', 'Week'],
                index=0
            )
            
            # Process data
            pivot_table, df_processed, time_column = process_data(df, time_period)
            
            if pivot_table.empty:
                st.warning(f"No valid data found for {time_period.lower()} analysis. Please check your {time_column} column format.")
                st.info(f"Expected format for {time_column}: {'Week format like \"Week 1 2024\"' if time_period == 'Week' else 'Month format like \"Jan 2024\"'}")
            else:
                # Calculate summary statistics
                status_totals = df['Booking Status'].value_counts()
                total_bookings = len(df)
                
                # Display summary statistics
                st.header("📈 Summary Statistics")
                col1, col2, col3, col4 = st.columns(4)
                
                with col1:
                    st.metric("Total Bookings", total_bookings)
                with col2:
                    confirmed = status_totals.get('Confirmed', 0)
                    confirmed_pct = (confirmed/total_bookings*100) if total_bookings > 0 else 0
                    st.metric("Confirmed", f"{confirmed}", f"{confirmed_pct:.1f}%")
                with col3:
                    cancelled = status_totals.get('Cancelled', 0)
                    cancelled_pct = (cancelled/total_bookings*100) if total_bookings > 0 else 0
                    st.metric("Cancelled", f"{cancelled}", f"{cancelled_pct:.1f}%")
                with col4:
                    rejected = status_totals.get('Rejected', 0)
                    rejected_pct = (rejected/total_bookings*100) if total_bookings > 0 else 0
                    st.metric("Rejected", f"{rejected}", f"{rejected_pct:.1f}%")
                
                # Create and display visualizations
                st.header("📊 Visualizations")
                fig = create_visualizations(pivot_table, df_processed, time_period)
                st.pyplot(fig)
                
                # Download Options
                st.header("💾 Download Options")
                col1, col2 = st.columns(2)
                
                with col1:
                    # Copy image button
                    img_base64 = fig_to_base64(fig)
                    st.markdown(
                        f"""
                        <button onclick="navigator.clipboard.writeText('data:image/png;base64,{img_base64}')">
                            📋 Copy Image to Clipboard
                        </button>
                        """,
                        unsafe_allow_html=True
                    )
                    
                    # Alternative: Display copyable base64 string
                    with st.expander("📋 Copy Image Data"):
                        st.text_area("Base64 Image Data (copy this):", 
                                   value=f"data:image/png;base64,{img_base64}", 
                                   height=100)
                
                with col2:
                    # PDF download button
                    pdf_data = create_pdf_download(fig)
                    st.download_button(
                        label="📄 Download as PDF",
                        data=pdf_data,
                        file_name=f"booking_analysis_{time_period.lower()}.pdf",
                        mime="application/pdf"
                    )
                
                # Display data table
                st.header("📋 Data Summary")
                st.dataframe(pivot_table, use_container_width=True)
                                    
    except Exception as e:
            st.error(f"An error occurred while processing the file: {str(e)}")
    else:
        st.info("Please upload a CSV file to get started")
